In [1]:
# 딱 한번만 돌릴것
from youtube_module import YoutubeBulider
# KEY 파일로 읽어오기
KEY_FILE = open('./API_KEY.txt', 'r')
# API_KEY 할당
API_KEY = KEY_FILE.readline()

# YoutbeModule 사용
bulider = YoutubeBulider(API_KEY)

# yotube_API를 이용한 인기 급상승 video_id 가져오기
bulider.get_videos()

# 인기 동영상의 video_id를 list로 가져온다.
video_list = bulider.get_video_list(10)

# video_list의 길이 :: top 몇 개
print(len(video_list))

10


In [ ]:
# 인기 동영상의 댓글 목록을 가져온다.
comments_list = bulider.get_comments(video_list)

# 댓글 목록을 DataFrame으로 만든다.
comments_df = bulider.create_df(comments_list)

# csv 파일로 저장하기 위함
comments_df.to_csv('./commets_df.csv')